In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_pretrain import simple_one_layer_cross_entropy_train
from fewshot.algorithms.fewshot_models import BaselineFewShotModel
from tensorflow.keras import callbacks

import numpy as np
import tqdm
import scipy.stats as st

import tensorflow as tf

from IPython.display import clear_output

### Init

Set up all input data and params

In [17]:
# common
seed = 11

img_width = 84
img_height = 84
img_depth = 3

# classes
base_num_classes = 64
novel_num_classes = 20
val_num_classes = 16  # not used in baseline algo

# backbone training
backbone_training_batch_size = 32
backbone_training_epochs = 2

backbone_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 25
query_samples_per_class = 16

support_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

n_episodes = 50

In [18]:
dataset_dir = "../../../data/mini_imagenet/"
dataset = Dataset(dataset_dir=dataset_dir, csv_name="data_84x84.csv", image_size=(img_width, img_height))

### Backbone
Create backbone dataset (80 classes), split to train and validattion

In [19]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 64 (seed = 11)
Train classes: 64
Test classes: 36
Train data: 38400 samples
Test data:  21600 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 16 (seed = 11)
Train classes: 16
Test classes: 20
Train data: 9600 samples
Test data:  12000 samples


Create classifier

In [6]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))

Instructions for updating:
Colocations handled automatically by placer.


Load weights if we have presaved

Train

In [7]:
backbone.set_trainable(True)

In [8]:
# TODO: incapsulate inside fewshot library
backbone_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("../../../data/few-models/weights.{epoch:02d}-{loss:.2f}.hdf5",
                                       monitor="loss",
                                       save_best_only=False)
]

In [ ]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    n_epochs=backbone_training_epochs,
    callbacks=backbone_callbacks
)

### Few Shot
Train fewshot model

In [ ]:
accuracies = []

episodes = fewshot_dataset.get_fewshot_generator(n_way=n_way, k_shot=k_shot, 
                                                 query_samples_per_class=query_samples_per_class,
                                                 support_generator_args=support_generator_args)
for episode in range(n_episodes):
    (support_x, support_y), (query_x, query_y) = next(episodes)
    
    fewshot_model = BaselineFewShotModel(backbone, n_way)
    fewshot_model.fit(support_x, support_y,
                      batch_size=fewshot_batch_size,
                      epochs=support_epochs,
                      optimizer="adam")

    # predict and calc accuracy
    out = fewshot_model.predict(query_x, batch_size=fewshot_batch_size)
    acc = np.mean(np.argmax(out, axis=1) == np.where(query_y == 1)[1])
    
    accuracies.append(acc)
    clear_output()

    print("Mean accuracy (by {} episodes): {:.2f}%".format(episode + 1, np.mean(accuracies) * 100))

### Report

Calc 95% confidence interval and report

In [11]:
acc_int = st.t.interval(0.95, len(accuracies) - 1, loc=np.mean(accuracies), scale=st.sem(accuracies))

In [12]:
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accuracies) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 44.22% +- 3.54%
